In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import cv2
import math
import os
from glob import glob
from scipy import stats as s

In [2]:
train = pd.read_csv('train_new.csv')
train = train.sample(n=10000).reset_index(drop=True)
train.head()

,image,class
0,v_IceDancing_g16_c01.avi_frame3.jpg,IceDancing
1,v_PlayingFlute_g12_c03.avi_frame3.jpg,PlayingFlute
2,v_BabyCrawling_g12_c02.avi_frame2.jpg,BabyCrawling
3,v_PlayingViolin_g15_c01.avi_frame0.jpg,PlayingViolin
4,v_Mixing_g22_c01.avi_frame1.jpg,Mixing


In [3]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [01:10<00:00, 142.84it/s]


(10000, 224, 224, 3)

In [4]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [5]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [6]:
# creating the base model of pre-trained VGG16 model
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

In [7]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(8000, 5, 5, 1536)

In [8]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(2000, 5, 5, 1536)

In [9]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(8000, 5*5*1536)
X_test = X_test.reshape(2000, 5*5*1536)

In [10]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [11]:
# shape of images
X_train.shape

(8000, 38400)

In [12]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(38400,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

In [13]:
# defining a function to save the weights of best model
from tensorflow.keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('InceptionResNetV2weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [14]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [15]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
63/63 [==============================] - 10s 157ms/step - loss: 4.6231 - accuracy: 0.0189 - val_loss: 4.4559 - val_accuracy: 0.0805
Epoch 2/200
63/63 [==============================] - 10s 154ms/step - loss: 4.3060 - accuracy: 0.0555 - val_loss: 3.9640 - val_accuracy: 0.1395
Epoch 3/200
63/63 [==============================] - 10s 152ms/step - loss: 3.9119 - accuracy: 0.1194 - val_loss: 3.3549 - val_accuracy: 0.2290
Epoch 4/200
63/63 [==============================] - 10s 153ms/step - loss: 3.4692 - accuracy: 0.1848 - val_loss: 2.9266 - val_accuracy: 0.2950
Epoch 5/200
63/63 [==============================] - 10s 155ms/step - loss: 3.0926 - accuracy: 0.2441 - val_loss: 2.4747 - val_accuracy: 0.4095
Epoch 6/200
63/63 [==============================] - 10s 155ms/step - loss: 2.7191 - accuracy: 0.3200 - val_loss: 2.1171 - val_accuracy: 0.4850
Epoch 7/200
63/63 [==============================] - 9s 150ms/step - loss: 2.4410 - accuracy: 0.3716 - val_loss: 1.9225 - val_accuracy: 

Epoch 58/200
63/63 [==============================] - 9s 137ms/step - loss: 0.4883 - accuracy: 0.8521 - val_loss: 0.6687 - val_accuracy: 0.8530
Epoch 59/200
63/63 [==============================] - 9s 138ms/step - loss: 0.4903 - accuracy: 0.8544 - val_loss: 0.6686 - val_accuracy: 0.8545
Epoch 60/200
63/63 [==============================] - 9s 138ms/step - loss: 0.4676 - accuracy: 0.8605 - val_loss: 0.6536 - val_accuracy: 0.8630
Epoch 61/200
63/63 [==============================] - 9s 140ms/step - loss: 0.4664 - accuracy: 0.8596 - val_loss: 0.6474 - val_accuracy: 0.8565
Epoch 62/200
63/63 [==============================] - 9s 140ms/step - loss: 0.4742 - accuracy: 0.8596 - val_loss: 0.6767 - val_accuracy: 0.8510
Epoch 63/200
63/63 [==============================] - 9s 141ms/step - loss: 0.4848 - accuracy: 0.8572 - val_loss: 0.6401 - val_accuracy: 0.8630
Epoch 64/200
63/63 [==============================] - 9s 140ms/step - loss: 0.4618 - accuracy: 0.8643 - val_loss: 0.6474 - val_accuracy:

Epoch 115/200
63/63 [==============================] - 9s 140ms/step - loss: 0.3164 - accuracy: 0.9100 - val_loss: 0.6442 - val_accuracy: 0.8735
Epoch 116/200
63/63 [==============================] - 9s 140ms/step - loss: 0.2989 - accuracy: 0.9172 - val_loss: 0.6725 - val_accuracy: 0.8645
Epoch 117/200
63/63 [==============================] - 9s 140ms/step - loss: 0.3076 - accuracy: 0.9153 - val_loss: 0.6606 - val_accuracy: 0.8710
Epoch 118/200
63/63 [==============================] - 9s 139ms/step - loss: 0.3023 - accuracy: 0.9160 - val_loss: 0.6470 - val_accuracy: 0.8715
Epoch 119/200
63/63 [==============================] - 9s 139ms/step - loss: 0.2808 - accuracy: 0.9230 - val_loss: 0.6777 - val_accuracy: 0.8670
Epoch 120/200
63/63 [==============================] - 9s 138ms/step - loss: 0.3148 - accuracy: 0.9172 - val_loss: 0.6627 - val_accuracy: 0.8660
Epoch 121/200
63/63 [==============================] - 9s 138ms/step - loss: 0.3125 - accuracy: 0.9120 - val_loss: 0.6391 - val_ac

63/63 [==============================] - 9s 138ms/step - loss: 0.2777 - accuracy: 0.9250 - val_loss: 0.6507 - val_accuracy: 0.8700
Epoch 172/200
63/63 [==============================] - 9s 139ms/step - loss: 0.2558 - accuracy: 0.9327 - val_loss: 0.6788 - val_accuracy: 0.8635
Epoch 173/200
63/63 [==============================] - 9s 139ms/step - loss: 0.2572 - accuracy: 0.9351 - val_loss: 0.7124 - val_accuracy: 0.8680
Epoch 174/200
63/63 [==============================] - 9s 137ms/step - loss: 0.2448 - accuracy: 0.9379 - val_loss: 0.7033 - val_accuracy: 0.8735
Epoch 175/200
63/63 [==============================] - 9s 138ms/step - loss: 0.2727 - accuracy: 0.9306 - val_loss: 0.7240 - val_accuracy: 0.8705
Epoch 176/200
63/63 [==============================] - 9s 140ms/step - loss: 0.2628 - accuracy: 0.9325 - val_loss: 0.7132 - val_accuracy: 0.8715
Epoch 177/200
63/63 [==============================] - 9s 139ms/step - loss: 0.2416 - accuracy: 0.9335 - val_loss: 0.7424 - val_accuracy: 0.8710

In [16]:
# loading the trained weights
model.load_weights("InceptionResNetV2weight.hdf5")

In [17]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [18]:
# getting the test list
f = open("ucfTrainTestlist/testlist02.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi


In [19]:
# creating the tags
train = pd.read_csv('train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [20]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('UCF-101/'+videoFile.split(' ')[0])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 5*5*1536)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])

  0%|                                                                                         | 0/3734 [00:00<?, ?it/s]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


100%|██████████████████████████████████████████████████████████████████████████████| 3734/3734 [37:07<00:00,  1.68it/s]


In [21]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

89.07337975361543

# Model Evaluation

Model performs the best among the evaluated base_models, this leaves more room for model tuning in order to improve performance.